In [1]:
import pandas as pd
train_df = pd.read_csv(r"D:\Projects\VsCode\Python\img_processing_system\practice\pytorch\data\otto_group_product\train.csv")
test_df = pd.read_csv(r"D:\Projects\VsCode\Python\img_processing_system\practice\pytorch\data\otto_group_product\test.csv")

In [2]:
import torch
from sklearn.feature_selection import SelectKBest
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
X = train_df[train_df.columns.difference(["id", "target"])]
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False).fit(train_df.iloc[:, 94:])
y = ohe.transform(train_df.iloc[:, 94:])
selector = SelectKBest(k=65).fit(X, LabelEncoder().fit_transform(train_df["target"])) # 再往上加特征数，模型的效果提升很小
selected_features = selector.get_feature_names_out()
X_selected_features_data = X.loc[:, selected_features]

X_tensor = torch.tensor(X_selected_features_data.values, dtype=torch.float32).to(device)
y_tensor = torch.tensor(y, dtype=torch.float32).to(device)


In [3]:
import torch.nn as nn

class Multiclass(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(len(selected_features), 40)
        self.act = nn.ReLU()
        self.output = nn.Linear(40, 9)
        
    def forward(self, x):
        x = self.act(self.hidden(x))
        x = self.output(x)
        return x
    
model = Multiclass()
model.to(device)

Multiclass(
  (hidden): Linear(in_features=65, out_features=40, bias=True)
  (act): ReLU()
  (output): Linear(in_features=40, out_features=9, bias=True)
)

In [4]:
import torch.optim as optim

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, train_size=0.7, shuffle=True)

# 问题：6m8s才完成42轮

In [ ]:
import copy
import numpy as np
# training parameters
best_acc = - np.inf 
best_weights = None
n_epochs = 10
# n_epochs = 200
batch_size = 5
batches_per_epoch = len(X_train) # batch_size
print(batches_per_epoch)
for epoch in range(n_epochs): 
    for i in range(0, batches_per_epoch, batch_size):
        # take a batch
        start = i
        X_batch = X_train[start:start+batch_size]
        y_batch = y_train[start:start+batch_size]
        # forward pass
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        # update weights
        optimizer.step()
    y_pred = model(X_test)
    ce = loss_fn(y_pred, y_test)
    acc = (torch.argmax(y_pred, 1) == torch.argmax(y_test, 1)).float().mean()
    if float(acc) > best_acc:
        best_acc = float(acc)
        best_weights = copy.deepcopy(model.state_dict())
    print(f"Epoch {epoch} validation: Cross-entropy={float(ce)}, Accuracy={float(acc)}")

model.load_state_dict(best_weights) # 留下最好的模型
print(best_acc)

43314


C:\Users\31204\AppData\Local\Temp\ipykernel_23028\1741193696.py:31: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\autograd\generated\python_variable_methods.cpp:836.)
  print(f"Epoch {epoch} validation: Cross-entropy={float(ce)}, Accuracy={float(acc)}")


Epoch 0 validation: Cross-entropy=0.6296306848526001, Accuracy=0.762389600276947
Epoch 1 validation: Cross-entropy=0.6083589196205139, Accuracy=0.7696078419685364
Epoch 2 validation: Cross-entropy=0.5977919697761536, Accuracy=0.7723012566566467
Epoch 3 validation: Cross-entropy=0.6003997921943665, Accuracy=0.7706313133239746
Epoch 4 validation: Cross-entropy=0.5999757051467896, Accuracy=0.7726244330406189
Epoch 5 validation: Cross-entropy=0.601300835609436, Accuracy=0.7727321982383728
Epoch 6 validation: Cross-entropy=0.6001300811767578, Accuracy=0.7747252583503723
Epoch 7 validation: Cross-entropy=0.5989786386489868, Accuracy=0.7749407291412354
Epoch 8 validation: Cross-entropy=0.6031131744384766, Accuracy=0.7743481993675232
Epoch 9 validation: Cross-entropy=0.6035953760147095, Accuracy=0.774025022983551
0.7749407291412354


In [7]:
X = test_df[test_df.columns.difference(["id"])]
X_selected_features_data = X.loc[:, selected_features]
X_tensor = torch.tensor(X_selected_features_data.values, dtype=torch.float32).to(device)

y_pred = model(X_tensor)
max_ele_idx = torch.argmax(y_pred, 1) # 获得每行最大的元素的下标
# 请根据max_ele_idx，生成一个与下面格式相同的csv文件，例如：max_ele_idx第一个元素的值是3，说明id=1时，Class_4=1，其余均为0；max_ele_idx第二个元素的值是7，说明id=2时，Class_8=1，其余均为0。
# id,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
# 1,1,0,0,0,0,0,0,0,0
# 2,1,0,0,0,0,0,0,0,0
# 先构造一个DataFrame，然后把toCSV()
sample_df = pd.read_csv(r"D:\Projects\VsCode\Python\img_processing_system\practice\pytorch\data\otto_group_product\sampleSubmission.csv")
res_df = pd.DataFrame(columns=sample_df.columns)
for i in range(1, len(y_pred) + 1):
    tmp_arr = [i, 0,0,0,0,0,0,0,0,0]
    tmp_arr[max_ele_idx[i-1].item() + 1] = 1
    res_df.loc[i-1] = tmp_arr
res_df.to_csv(r"D:\Projects\VsCode\Python\img_processing_system\practice\pytorch\data\otto_group_product\result.csv", index= False)